In [1]:
import pandas as pd
import datetime
import os
from haversine import haversine
import json
import numpy as np
zip_centers=json.load(open("/home/jian/Docs/Geo_mapping/updated_zip_centers_JL_2019-05-23.json","r"))

os.getcwd()

'/home/jian/Projects/Big_Lots/Analysis/2019_Q3/Planner_request/Faith_zips_for_SOTF_5211_20190911'

In [2]:
zip_labels=pd.ExcelFile("/home/jian/Projects/Big_Lots/New_TA/TA_created_in_201906/final_output_20190718/BL_final_TA_updated_remove_8_and_22_stores_JL_2019-09-11.xlsx")
zip_labels.sheet_names

['unique_zips_full_footprint', 'view_by_TA', 'view_by_store', 'stores_removed']

In [3]:
df=zip_labels.parse("view_by_store",dtype=str)
df=df[df['location_id']=="5211"]
df['all_trans_P_zips']=df['all_trans_P_zips'].apply(eval)
df['all_trans_S_zips']=df['all_trans_S_zips'].apply(eval)

df.shape


(1, 17)

In [13]:
df

,TA_num,ta_name,location_id,address_line_1,address_line_2,zip_cd,city_nm,state_nm,latitude_meas,longitude_meas,DMA,CTY,all_trans_P_zips,all_trans_S_zips,zips_in_10,trans_P_zips_70_within_TA,trans_S_zips_70_within_TA
1357,686,single_5211,5211,1650 E. SHERMAN BLVD,nan,49444,MUSKEGON,MI,43.206841,-86.201683,{'GRAND RAPIDS-KALMZOO-B.CRK'},{'MUSKEGON'},"[49442, 49441, 49444]","[49445, 49457, 49456, 49417, 49415]",[],"['49442', '49444', '49441']","['49456', '49417', '49457', '49445', '49415']"


In [4]:
list_P_zips=df['all_trans_P_zips'].values[0]
df_P_zips=pd.DataFrame({"zip":list_P_zips},index=['5211']*len(list_P_zips)).reset_index().rename(columns={"index":"location_id"})
df_P_zips['transaction_flag']="P"
list_S_zips=df['all_trans_S_zips'].values[0]
df_S_zips=pd.DataFrame({"zip":list_S_zips},index=['5211']*len(list_S_zips)).reset_index().rename(columns={"index":"location_id"})
df_S_zips['transaction_flag']="S"
df_P_S_zips=df_P_zips.append(df_S_zips)

In [5]:
store_list=pd.read_table("/home/jian/BigLots/static_files/Store_list/MediaStormStores20190901-135942-508.txt",dtype=str,sep="|")
store_list=store_list[store_list['location_id']=="5211"]
store_list=store_list[['location_id','zip_cd','latitude_meas','longitude_meas','state_nm']].rename(columns={"zip_cd":"store_zip"})
store_list['latitude_meas']=store_list['latitude_meas'].astype(float)
store_list['longitude_meas']=store_list['longitude_meas'].astype(float)
store_list['store_zip']=store_list['store_zip'].apply(lambda x: x.split("-")[0].zfill(5))

In [6]:
df_output=pd.merge(store_list,df_P_S_zips,on="location_id",how="left")

In [8]:
df_P_S_zips.shape

(8, 3)

In [9]:
# Check location
store_list=pd.read_table("/home/jian/BigLots/static_files/Store_list/MediaStormStores20190901-135942-508.txt",dtype=str,sep="|")
store_list=store_list[store_list['location_id']=="5211"]
store_list

,location_id,location_desc,open_dt,close_dt,address_line_1,address_line_2,city_nm,state_nm,zip_cd,longitude_meas,latitude_meas,sales_square_feet,furniture_sales_square_feet
1230,5211,SHERMAN CENTRE,2011-10-30,NaN,1650 E. SHERMAN BLVD,NaN,MUSKEGON,MI,49444-1863,-000000000086.201683,000000000043.206841,00000000019407.0000,00000000002075.0000


In [10]:
df_output=df_output.reset_index()
del df_output['index']
df_output['Distance (mile)']=np.nan
for ind,row in df_output.iterrows():
    store_center=(row['latitude_meas'],row['longitude_meas'])
    dist=haversine(store_center,zip_centers[row['zip']],miles=True)
    df_output.loc[ind,"Distance (mile)"]=dist

In [12]:
writer=pd.ExcelWriter("./BL_zips_for_store_5211_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
df_output.to_excel(writer,"zips_5211",index=False)
writer.save()